In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
AI_STUDIO_KEY = os.getenv("GOOGLE_AI_STUDIO_KEY")

## Connect

In [ ]:
import weaviate

# Connect to the local instance deployed with Docker Compose
client = weaviate.connect_to_local(
    headers={
       "X-Google-Api-Key": AI_STUDIO_KEY,
    },

    # Set longer timeout
    additional_config=weaviate.config.AdditionalConfig(timeout=(500, 1500))
)

client.is_ready()

In [ ]:
# import weaviate, os

# client = weaviate.connect_to_wcs(
#     cluster_url=os.getenv("WORKSHOP_DEMO_URL"),
#     auth_credentials=weaviate.auth.AuthApiKey(os.getenv("WORKSHOP_DEMO_KEY_READ")),

#     headers={
#        "X-Palm-Api-Key": AI_STUDIO_KEY,
#     },

#     # Set longer timeout
#     additional_config=weaviate.config.AdditionalConfig(timeout=(500, 1500))
# )

# # client.is_ready()

## RAG - single prompt

In [ ]:
papers = client.collections.get("Papers")

response = papers.query.near_text("challenges of zero shot training", limit=5)

for item in response.objects:
    print(item.properties["chunk"])

In [ ]:
papers = client.collections.get("Papers")

response = papers.generate.near_text(
    "challenges of zero shot training",
    limit=5,
    single_prompt="What are the challenges of zero shot training? Please answer based on the following chunk: {chunk}"
)

for item in response.objects:
    print(item.properties["chunk"])
    print(item.generated)
    print("--------------------\n")

In [ ]:
papers = client.collections.get("Papers")

response = papers.generate.near_text(
    "zero shot training",
    limit=5,
    single_prompt="Please explain what the following chunk of text talks about in the context of zero-shot training: {chunk}"
)

for item in response.objects:
    print(item.properties["chunk"])
    print(item.generated)
    print("----------------------------------------\n")

## RAG - group task

In [ ]:
papers = client.collections.get("Papers")

response = papers.generate.near_text(
    "difficulties and challenges of zero shot training",
    limit=5,
    grouped_task="What can you tell me about zero shot training? Please answer based on the provided content",
)

for item in response.objects:
    print(item.properties["chunk"])
    
print("----------------------------------------\n")
print(response.generated)

In [ ]:
papers = client.collections.get("Papers")

response = papers.generate.near_text(
    "challenges of zero shot training",
    limit=5,
    grouped_task="What can you tell me about zero shot training? Please answer based on the provided content",
    grouped_properties=["chunk", "title"]
)

for item in response.objects:
    print(item.properties["chunk"])
    
print("----------------------------------------")
print(response.generated)

## Close the client when done

In [ ]:
client.close()